<a href="https://colab.research.google.com/github/Souviksamanta34/Biomarker_Identification_for_Cancer_Detection/blob/main/new01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import tensorflow as tf
import numpy as np
import random
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from keras.layers import GaussianNoise
from keras.callbacks import EarlyStopping
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from keras import regularizers
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, GaussianNoise
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from tensorflow.keras.models import Model
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from google.colab import drive



class AutoEncoder:
    def __init__(self, input_dim, latent_dim):
        inputs = Input(shape=(input_dim,))
        encoded = Dense(128, activation='relu',activity_regularizer=regularizers.l1(10e-5))(inputs)
        encoded = Dropout(0.5)(encoded)
        latent = Dense(latent_dim, activation='relu')(encoded)
        decoded = Dense(128, activation='relu')(latent)
        decoded = Dropout(0.5)(decoded)
        outputs = Dense(input_dim, activation='sigmoid')(decoded)
        self.autoencoder = Model(inputs, outputs)
        self.encoder = Model(inputs, latent)
        self.autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

class TriphasicDeepBRCA:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.autoencoder = AutoEncoder(input_dim=X.shape[1], latent_dim=128)
        self.neural_network = Sequential()
        self.neural_network.add(Dense(units=256, activation='relu', input_shape=(128,)))
        self.neural_network.add(Dense(units=128, activation='relu'))
        self.neural_network.add(Dense(units=64, activation='relu'))
        self.neural_network.add(Dense(units=len(np.unique(y)), activation='softmax'))
        self.svm = SVC(kernel='rbf')

    def train(self):
        X_encoded = self.autoencoder.encoder.predict(self.X)
        self.neural_network.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        self.neural_network.fit(X_encoded, self.y, epochs=100)
        def RFE_model(X_encoded, y):
            estimator = RandomForestClassifier()
            selector = RFE(estimator, n_features_to_select=85, step=1)
            selector = selector.fit(X_encoded, y)
            biomarker_genes = np.where(selector.support_)[0]
            return biomarker_genes

        biomarker_genes = RFE_model(X_encoded, self.y)
        with open('biomarker_genes.txt', 'w') as f:
            for gene in biomarker_genes:
                f.write(str(gene) + '\n')

random.seed(0)
np.random.seed(0)
drive.mount('/content/drive')
# Get the path to the file
file_path = '/content/drive/MyDrive/Dataset/FinalBRCA.csv'

# Define the classifiers
classifiers = [
   ('rf', RandomForestClassifier()),
   ('svc', SVC(probability=True)),
   ('knn', KNeighborsClassifier()),
   ('dt', DecisionTreeClassifier())
]

# Define the ensemble methods
ensemble_methods = [
   ('voting', VotingClassifier(estimators=classifiers)),
   ('bagging', BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=10)),
   ('adaboost', AdaBoostClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=10))
]

# Define the hyperparameters to tune
hyperparameters = {
   'rf__n_estimators': [10, 50, 100, 200],
   'rf__max_depth': [None, 10, 20, 30],
   'svc__C': [0.1, 1, 10, 100],
   'svc__gamma': [1, 0.1, 0.01, 0.001],
   'knn__n_neighbors': [3, 5, 7, 9],
   'dt__max_depth': [None, 10, 20, 30]
}



encoder = LabelEncoder()
scaler = StandardScaler()
smote = SMOTE()
# Load the data from a CSV file
data = pd.read_csv(file_path)
for column in data.columns:
    data[column] = data[column].astype(str)
for column in data.columns:
    data[column] = encoder.fit_transform(data[column])
y_train = data['PAM50']
y_train_encoded = encoder.fit_transform(y_train)
y_train_encoded = to_categorical(y_train_encoded)
X_train = data.drop(columns=['sample','PAM50'])
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
y_test_encoded = encoder.fit_transform(y_test)
y_test_encoded = to_categorical(y_test_encoded)
X_train = scaler.fit_transform(X_train)
X_train, y_train = smote.fit_resample(X_train, y_train)
model = TriphasicDeepBRCA(X_train, y_train)
model.train()

# Load the data from a CSV file
data = pd.read_csv(file_path)

for column in data.columns:
    data[column] = data[column].astype(str)

# Get the name of biomarker genes
with open('biomarker_genes.txt', 'r') as f:
    biomarker_genes_index = [int(line.strip()) for line in f]

# Check if indices are within range
biomarker_genes_index = [i for i in biomarker_genes_index if i < len(data.columns)]

biomarker_genes = data.columns[biomarker_genes_index]
print('Biomarker genes:', biomarker_genes)

# Create a mask for the biomarker genes (including the 'Label' column)
mask = data.columns.isin(biomarker_genes) | (data.columns == 'PAM50')

# Use the mask to select only the columns that correspond to the biomarker genes and the 'Label' column
data_biomarker = data.loc[:, mask]

# Separate features and target
X = data_biomarker.drop(['PAM50'], axis=1)
y = data_biomarker['PAM50']

# Identify the column that starts with 'TCGA'
column_to_drop = X.columns[X.apply(lambda x: x.str.startswith('TCGA').any())]

# Drop the identified column
X = X.drop(column_to_drop, axis=1)

# Scale the features
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an imputer object
imp = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit and transform the data
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

X_train, y_train = smote.fit_resample(X_train, y_train)

print('blaaaaaaaaaaa')

# Perform hyperparameter tuning
for method in ensemble_methods:
   if method[0] == 'voting':
      clf = RandomizedSearchCV(method[1], hyperparameters, cv=5)
      clf.fit(X_train, y_train)
      y_pred = clf.predict(X_test)
      print(f'Accuracy for {method[0]}: {accuracy_score(y_test, y_pred)}')
   else:
      clf = method[1]
      clf.fit(X_train, y_train)
      y_pred = clf.predict(X_test)
      print(f'Accuracy for {method[0]}: {accuracy_score(y_test, y_pred)}')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
68/68 [==============================] - 1s 8ms/step
Epoch 1/100
68/68 [==============================] - 1s 3ms/step - loss: 0.9231 - accuracy: 0.6633
Epoch 2/100
68/68 [==============================] - 0s 3ms/step - loss: 0.4225 - accuracy: 0.8393
Epoch 3/100
68/68 [==============================] - 0s 3ms/step - loss: 0.2818 - accuracy: 0.8984
Epoch 4/100
68/68 [==============================] - 0s 3ms/step - loss: 0.1783 - accuracy: 0.9395
Epoch 5/100
68/68 [==============================] - 0s 3ms/step - loss: 0.1303 - accuracy: 0.9575
Epoch 6/100
68/68 [==============================] - 0s 3ms/step - loss: 0.0907 - accuracy: 0.9718
Epoch 7/100
68/68 [==============================] - 0s 3ms/step - loss: 0.0685 - accuracy: 0.9792
Epoch 8/100
68/68 [==============================] - 0s 3ms/step - loss: 0.0305 - accuracy: 0.9931
Epoch 9/100
68/68 [=======

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy for bagging: 0.7956521739130434


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Accuracy for adaboost: 0.6782608695652174
